In [5]:
import pandas as pd 
import torch 
import torchvision
import numpy as np
import matplotlib.pyplot as plt 
import seaborn  as sns 
from torch import nn

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
from torchvision import transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.ToTensor(),
    
])
train_data = torchvision.datasets.Food101(root="data" , 
                                          split="train" , 
                                          transform= train_transform)

test_data = torchvision.datasets.Food101(root="data" , 
                                         split="test" ,
                                         transform=train_transform)

In [7]:
model = torchvision.models.resnet50("IMAGENET1K_V2")


c:\Users\Vedant\anaconda3\envs\Pytorch_cuda_venv\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [8]:
for parameters in model.parameters() :
    parameters.requires_grad = False 

In [9]:
num_classes = 101
model.fc = nn.Linear(model.fc.in_features , num_classes)

In [15]:
for name , parameters in model.named_parameters() :
    if name == "fc" :
        parameters.requires_grad = True

In [21]:
## Setting up the data loaders 
from torch.utils.data import DataLoader
train_data_loader = DataLoader(dataset=train_data ,
                               batch_size=32 , 
                               shuffle= True , 
                               pin_memory=True , 
                               num_workers=6)
test_data_loader = DataLoader(dataset=test_data ,
                               batch_size=32 , 
                               shuffle= True , 
                               pin_memory=True , 
                               num_workers=6)

In [23]:
## Setting up the loss function and optimizer  and accuracy fn 
from torchmetrics import Accuracy

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=model.fc.parameters() , lr = 1e-3)

accuracy_fn = Accuracy(task="multiclass" ,num_classes=101).to(device)



In [28]:
from tqdm import tqdm
def Train_loop(model, data_loader, accuracy_fn, optimizer, loss_fn, device):
    model.train()
    model.to(device)

    accuracy_fn.reset()
    train_loss = 0.0

    for images, labels in tqdm(data_loader):
        images, labels = images.to(device), labels.to(device)

        logits = model(images)
        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        preds = logits.argmax(dim=1)
        accuracy_fn.update(preds, labels)

    train_loss /= len(data_loader)
    train_acc = accuracy_fn.compute().item()

    print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}")
        
def Test_loop(model, data_loader, loss_fn, accuracy_fn, device, max_batches=None):
    model.eval()
    accuracy_fn.reset()
    test_loss = 0.0

    with torch.no_grad():
        for batch_idx, (X, y) in tqdm(enumerate(data_loader)):
            if max_batches is not None and batch_idx >= max_batches:
                break

            X, y = X.to(device), y.to(device)

            logits = model(X)
            loss = loss_fn(logits, y)

            test_loss += loss.item()
            preds = logits.argmax(dim=1)
            accuracy_fn.update(preds, y)

    test_loss /= (batch_idx + 1)
    test_acc = accuracy_fn.compute().item()

    print(f"Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

    return test_loss, test_acc



## Training the Classifier layer

In [24]:
from tqdm import tqdm
epochs = 10
for epoch in range(epochs):
    print(f"Epoch : {epoch}")
    Train_loop(
        model=model,
        data_loader=train_data_loader,
        loss_fn=loss_fn ,
        optimizer=optimizer ,
        device=device,
        accuracy_fn=accuracy_fn ,
    )
    Test_loop(
        model=model, 
        data_loader=test_data_loader,
        loss_fn=loss_fn ,
        accuracy_fn=accuracy_fn ,
        device=device  ,
        max_batches=50
    )
    


Epoch : 0


100%|██████████| 2368/2368 [03:05<00:00, 12.78it/s]


Train loss: 2.3307 | Train acc: 0.4723
Test loss: 1.5152 | Test acc: 0.6131
Epoch : 1


100%|██████████| 2368/2368 [03:09<00:00, 12.48it/s]


Train loss: 1.5559 | Train acc: 0.6154
Test loss: 1.3576 | Test acc: 0.6350
Epoch : 2


...

Train loss: 1.1184 | Train acc: 0.7096
Test loss: 1.2451 | Test acc: 0.6675
Epoch : 5


100%|██████████| 2368/2368 [03:05<00:00, 12.80it/s]


Train loss: 1.0420 | Train acc: 0.7267
Test loss: 1.2667 | Test acc: 0.6600
Epoch : 6


  0%|          | 0/2368 [00:06<?, ?it/s]


KeyboardInterrupt: 

## Training the layer 4 of the model 

In [26]:
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


In [27]:
## Setting up the new optmizer 
optimizer = torch.optim.AdamW(
    [
        {"params": model.layer4.parameters(), "lr": 1e-4},
        {"params": model.fc.parameters(),     "lr": 1e-3},
    ],
    weight_decay=1e-4
)


In [25]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="max",        # because we monitor accuracy
    patience=3,        # wait 3 epochs with no improvement
    factor=0.3,        # reduce LR to 30%
)


In [29]:
%%time
best_acc = 0.0

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    Train_loop(
        model=model,
        data_loader=train_data_loader,
        accuracy_fn=accuracy_fn,
        optimizer=optimizer,
        loss_fn=loss_fn,
        device=device
    )

    val_loss, val_acc = Test_loop(
        model=model,
        data_loader=test_data_loader,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device=device , 
        max_batches = 50
        
    )


    scheduler.step(val_acc)


    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")
        print("Saved new best model")



Epoch 1/10


100%|██████████| 2368/2368 [03:38<00:00, 10.83it/s]


Train loss: 0.8697 | Train acc: 0.7646


50it [00:03, 13.20it/s]


Test loss: 0.8442 | Test acc: 0.7650
Saved new best model

Epoch 2/10


...

Train loss: 0.0856 | Train acc: 0.9770


50it [00:03, 13.40it/s]


Test loss: 1.0420 | Test acc: 0.7506

Epoch 5/10


  0%|          | 0/2368 [00:03<?, ?it/s]


KeyboardInterrupt: 

## Training the layer 3 for the epoch 

In [34]:
model = torchvision.models.resnet50(weights="IMAGENET1K_V2")
model.fc = torch.nn.Linear(model.fc.in_features, 101)

model.load_state_dict(torch.load("best_model.pt"))
model.to(device)
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [37]:
optimizer = torch.optim.AdamW(
    [
        {"params": model.layer3.parameters(), "lr": 1e-5},
        {"params": model.layer4.parameters(), "lr": 3e-5},
        {"params": model.fc.parameters(),     "lr": 3e-4},
    ],
    weight_decay=1e-4
)


In [38]:
%%time
best_acc = 0.0

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    Train_loop(
        model=model,
        data_loader=train_data_loader,
        accuracy_fn=accuracy_fn,
        optimizer=optimizer,
        loss_fn=loss_fn,
        device=device
    )

    val_loss, val_acc = Test_loop(
        model=model,
        data_loader=test_data_loader,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device=device , 
        max_batches = 50
        
    )


    scheduler.step(val_acc)


    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")
        print("Saved new best model")



Epoch 1/10


100%|██████████| 2368/2368 [07:51<00:00,  5.02it/s]


Train loss: 0.2364 | Train acc: 0.9399


50it [00:03, 13.44it/s]


Test loss: 0.8382 | Test acc: 0.7681
Saved new best model

Epoch 2/10


...

Train loss: 0.0415 | Train acc: 0.9933


50it [00:03, 13.93it/s]


Test loss: 0.8755 | Test acc: 0.7731
Saved new best model

Epoch 4/10


 14%|█▍        | 330/2368 [01:26<08:52,  3.83it/s] 


KeyboardInterrupt: 

In [39]:
Test_loop(
        model=model,
        data_loader=test_data_loader,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device=device , 
        
    )

790it [00:51, 15.24it/s]

Test loss: 0.9655 | Test acc: 0.7740


(0.9655195226586317, 0.7739802002906799)